# Web Scrape labelled images of building styles

# Toronto Plaques
* http://torontoplaques.com/Menu_Subjects.html

In [1]:
# Import libaries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import string
import re
import urllib
import os
import time
from datetime import datetime
import string

In [2]:
from sqlalchemy import create_engine, Column, Integer, String, Sequence, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from models import connect_db, PointsOfInterest, ArchitecturalStyles, Architects,POICategories

### Set up base variables

In [3]:
rerun_webscrape=False # rerun all  webscraping
populate_db = False # repopulate database

# search by Builidng Type
main_page = 'http://www.cabbagetownpeople.ca/biographies/plaquees/'
#style_url="http://www.acotoronto.ca/search_buildingsDB_d2.php"
#site_root = "http://torontoplaques.com"
debug=False
plaques_list = []

df_to_db_map={
    'Name':'name',
    'Street' :'address' ,
    'URL':'external_url',
    'Notes': 'details',
}

In [2]:
def load_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Error connecting: status code {response.status_code}")

In [6]:
def load_person_page(person_url):
    #location_text= ""
    plaque_text = ""
    plaque_dict = {
        'Name':None,
        'URL': person_url,
        'Notes':None,
        'Street': None,
        'Description': None
    }

   # person_url ="http://www.cabbagetownpeople.ca/person/walter-seymour-allward/"
    html = load_page(person_url)
    soup = BeautifulSoup(html)
    start=soup.find('div', {'class':'entry-content'})
    for tag in start.recursiveChildGenerator():
        if 'Plaque located at' in tag:
            plaque_dict['Street'] = tag.replace('Plaque located at: ','')
        elif tag.name=='h3':
            plaque_dict['Name'] = tag.text.strip()
        elif tag.name=='h4':
            plaque_dict['Description'] = tag.text.strip()
        elif tag.name=='p':
            plaque_text+= tag.text.strip()

    plaque_dict['Notes'] = plaque_text

    return plaque_dict


In [8]:
def run_webscrape():
    html = load_page(main_page)
    soup = BeautifulSoup(html)
    subjects = soup.find('ul', {'id':'posts'}).findChildren('a')
    for subject in subjects:
        asubj = subject.text.strip()
        subject_url = subject.get_attribute_list('href')[0]
        print(subject_url)
        if subject_url != None:
            per_dict =load_person_page(subject_url)
            plaques_list.append(per_dict)
        time.sleep(5)
#     plq_df = pd.DataFrame(plaques_list)
#     plq_df.to_csv('../data/cabbagetown_plaques_' + str(round(time.time(),0)) + '.csv')


None
http://www.cabbagetownpeople.ca/person/test-person-1/
None
http://www.cabbagetownpeople.ca/person/walter-seymour-allward/
http://www.cabbagetownpeople.ca/person/betty-oliphant/
http://www.cabbagetownpeople.ca/person/allen-yen/
http://www.cabbagetownpeople.ca/person/mary-oriordan/
http://www.cabbagetownpeople.ca/person/roger-abbott/
None
http://www.cabbagetownpeople.ca/person/gordon-sinclair/
http://www.cabbagetownpeople.ca/person/patricia-watson/
http://www.cabbagetownpeople.ca/person/allan-king/
http://www.cabbagetownpeople.ca/person/lieutenant-francis-gwillim-simcoe/
http://www.cabbagetownpeople.ca/person/charles-sauriol/
http://www.cabbagetownpeople.ca/person/oronhyatekha-m-d/
http://www.cabbagetownpeople.ca/person/rowena-hume/
http://www.cabbagetownpeople.ca/person/sarah-anne-curzon/
http://www.cabbagetownpeople.ca/person/benjamin-brick/
http://www.cabbagetownpeople.ca/person/cabbagetowns-first-people/
None
http://www.cabbagetownpeople.ca/person/lister-sinclair/
None
http://ww

In [15]:
if rerun_webscrape:
    run_webscrape()
    bld_df = pd.DataFrame(plaques_list)
    bld_df.to_csv('../data/cabbagetown_plaques_'+ str(round(time.time(),0)) + '.csv')
else:
    bld_df=pd.read_csv('../data/cabbagetown_plaques_1543031716.0.csv',index_col=0)
    # merge description and notes
    bld_df['Notes'] = bld_df['Description'] + ".  " + bld_df['Notes']
bld_df.head()    

,Description,Name,Notes,Street,URL
0,"Writer, Self-trained Biologist, Scientific Ill...",Ernest Thompson Seton 1860 - 1946,"Writer, Self-trained Biologist, Scientific Ill...","6 Aberdeen Avenue, Toronto, ON, Canada",http://www.cabbagetownpeople.ca/person/test-pe...
1,Renowned Canadian Sculptor,"Walter Seymour Allward, R.C.A. 1876 - 1955",Renowned Canadian Sculptor. Walter Seymour Al...,"43 Amelia Street, Toronto, ON, Canada",http://www.cabbagetownpeople.ca/person/walter-...
2,World-Renowned Innovator of Ballet Education,Betty Oliphant 1918 - 2004,World-Renowned Innovator of Ballet Education. ...,"137 Amelia Street, Toronto, ON, Canada",http://www.cabbagetownpeople.ca/person/betty-o...
3,A Leading Figure in Canadian Radio Astronomy,"Allen Yen, PhD 1925 - 1993",A Leading Figure in Canadian Radio Astronomy. ...,"139 Amelia Street, Toronto, ON, Canada",http://www.cabbagetownpeople.ca/person/allen-yen/
4,Pioneering Veterinarian,"Dr. Mary O'Riordan, D.V.M. 1925 - 1993","Pioneering Veterinarian. Born in Ennis, Count...","160 Amelia Street, Toronto, ON, Canada",http://www.cabbagetownpeople.ca/person/mary-or...


In [25]:
def save_to_database_ORM(session, df):
    '''
    Saves scraped data to database using SqlAlchemy ORM
    Updates three tables: points_of_interest, archtectural_styles, architects
    The relationship between these tables is defined in models.py, so it automatically populates the poi_id column
    in the child tables with the poi_id of the main entry 
    '''
    
    for index, row in df.iterrows():
        
        poi_dict ={df_to_db_map[k]:v for k, v in row.items() if k in df_to_db_map.keys() and not pd.isnull(v)}
        poi_dict['poi_type']="Plaque"
        # hardcode type to statue 

        poi = PointsOfInterest(**poi_dict )

        session.add(poi)
        session.commit()

In [26]:

if populate_db:
    db=connect_db() #establish connection
    Session = sessionmaker(bind=db)
    session = Session() 
    save_to_database_ORM(session, bld_df)

In [27]:
session.query(PointsOfInterest).count()


6230